# 06 Recommendation Systems
__Math 3280 - Data Mining__ : Snow College : Dr. Michael E. Olson

* Leskovec, Chapter 9
-----

In [2]:
import numpy as np
import pandas as pd

### User Profiles

In [65]:
## Item (Movie) Profile

movies = ['M01','M02','M03','M04','M05','M06','M07','M08','M09','M10','M11','M12','M13','M14','M15']
actors = {
    'movies' : ['M01','M02','M03','M04','M05','M06','M07','M08','M09','M10','M11','M12','M13','M14','M15'],
    'Julia Roberts': [1,0,1,0,0,0,1,1,0,1,1,0,0,0,0]
}

movie_profile = pd.DataFrame(actors)
#movie_profile.columns = movies
movie_profile.set_index('movies', inplace=True)

movie_profile

,Julia Roberts
movies,
M01,1
M02,0
M03,1
M04,0
M05,0
M06,0
M07,1
M08,1
M09,0


In [67]:
## User Rating = Utility Matrix

ratings = {
    'movies' : ['M01','M02','M03','M04','M05','M06','M07','M08','M09','M10','M11','M12','M13','M14','M15'],
    'User A': [0,1,1,1,0,1,1,0,1,0,0,1,1,1,1]
}

user_ratings = pd.DataFrame(ratings)
#user_ratings.columns = movies
user_ratings.set_index('movies', inplace=True)

user_ratings

,User A
movies,
M01,0
M02,1
M03,1
M04,1
M05,0
M06,1
M07,1
M08,0
M09,1


In [69]:
## Example 9.3 : Movies given only likes or dislikes
user_Julia = np.dot(user_ratings['User A'], movie_profile['Julia Roberts'])
user_likes = np.dot(user_ratings['User A'], user_ratings['User A'])
print(f"Dot Product of User Likes and Actor's Movies : {user_Julia}")
print(f"Norm of User Likes (number of movies liked)  : {user_likes}")
print(f"User weight to movies with Julia Roberts     : {user_Julia / user_likes}")

Dot Product of User Likes and Actor's Movies : 2
Norm of User Likes (number of movies liked)  : 10
User weight to movies with Julia Roberts     : 0.2


In [70]:
## User Rating = Utility Matrix
## Rating of 1-5 stars : 0 means no rating

ratings = {
    'movies' : ['M01','M02','M03','M04','M05','M06','M07','M08','M09','M10','M11','M12','M13','M14','M15'],
    'User A': [0,5,3,4,1,3,5,0,4,0,2,5,5,3,4]
}

user_ratings = pd.DataFrame(ratings)
#user_ratings.columns = movies
user_ratings.set_index('movies', inplace=True)

user_ratings

,User A
movies,
M01,0
M02,5
M03,3
M04,4
M05,1
M06,3
M07,5
M08,0
M09,4


In [75]:
## Show only the ratings for movies with Julia Roberts
user_ratings[user_ratings['User A'] * movie_profile['Julia Roberts'] > 0]

,User A
movies,
M03,3
M07,5
M11,2


In [76]:
## Ratings - average rating
user_ratings[user_ratings['User A'] * movie_profile['Julia Roberts'] > 0] - user_ratings['User A'].mean()

,User A
movies,
M03,0.066667
M07,2.066667
M11,-0.933333
